# A Beginner's Guide to ConnectX
Taking part in your first Kaggle competition is hard. Often we have a solution on our computer that works perfectly yet our submission is marked as having "failed". This guide should help any new Kagglers to make their first submission for the ConnectX challenge. Feel free to copy and edit this workbook to obtain an average score in the current leaderboard :)

# The First Step
First, we have to decide to make a script or notebook in Kaggle. Submissions can be made in both but we can only observe HTML elements in notebooks. This means that to render games, we need to use a notebook.

Next, we need to link this notebook to the competition dataset. Open the right-hand panel on your screen and find the data tab. Click add data and search for your competition under the competition tab. Here, we want "ConnectX". Add the data and we are set-up!

# Implement the game environment
We want to do two things in this notebook:
1. Test and edit our agents
2. Submit our solution

To test and edit our agents, we have to set up the test environment as shown in the Reinforcement Learning course.

In [ ]:
# Here we import standard libraries and our environment
# You must first add the data for the task in the settings column
import random
import numpy as np
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# List the available agents
print(list(env.agents))

Let's check this works by running two random agents against each other and rendering to watch the game.

In [ ]:
env.run(["random", "random"])

# To render using iPython, we have to use a notebook as the Kaggle editor can't show HTML objects
env.render(mode="ipython")

# Making Our First Agents
The next few sections will closely follow the structure of the Reinforcement Learning course and can be skipped at your convenience.

Let's create three basic agents:
1. An agent who chooses to place his piece randomly
2. An agent who always chooses the middle column (whether or not it's valid)
3. An agent who always chooses the leftmost column

The competition will accept any function name so you do not need to use "my_agent" as in the course.

In [ ]:
# Agent 1: Random
def agent_random(obs, config):
    import random
    valid_moves = [col for col in range(config.columns) if obs.board[col]==0]
    return random.choice(valid_moves)

# Agent 2: Middle
def agent_middle(obs, config):
    return config.columns//2

# Agent 3: Leftmost
def agent_leftmost(obs, config):
    valid_moves = [col for col in range(config.columns) if obs.board[col]==0]
    return valid_moves[0]

# Let's Play
We can now run two of these agents against each other and observe the match within our notebook.

In [ ]:
# Choosing the random and leftmost agents
env.run([agent_random, agent_leftmost])
env.render(mode="ipython")

# Winning Percentages
We have some hidden code to determine the percentage of wins from 100 rounds of the game

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

In [ ]:
# Compute the winning percentages
# Commented out as it takes a while
# get_win_percentages(agent1=agent_random, agent2=agent_random)

# Check for a winning move
Directly copied from the course, here we write a function that determines whether a winning move is possible. If we can win of the next move, our agent makes that move.

In [ ]:
# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid

# Returns True if dropping piece in column results in game win
def check_winning_move(obs, config, col, piece):
    import numpy as np
    
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    next_grid = drop_piece(grid, col, piece, config)
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[row,col:col+config.inarow])
            if window.count(piece) == config.inarow:
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(next_grid[row:row+config.inarow,col])
            if window.count(piece) == config.inarow:
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(next_grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if window.count(piece) == config.inarow:
                return True
    return False

# A smarter agent
Let's create an agent that will always take the winning move if available. The agent (called blocker) will also observe if the other opponent can win of the next move and block accordingly.

In [ ]:
def agent_blocker(obs, config):
    # This function will be our submission so must be self-contained
    import random
    
    # Produce a list of valid moves
    valid_moves = [col for col in range(config.columns) if obs.board[col] == 0]
    
    # If we have a winning move, take it
    for col in valid_moves:
        if check_winning_move(obs, config, col, obs.mark):
            return col 
    
    # Else if the opponent has a winning move, block it
    opponent = (obs.mark%2)+1
    for col in valid_moves:
        if check_winning_move(obs, config, col, opponent):
            return col
    
    # Else return a random move
    return random.choice(valid_moves)

In [ ]:
# Run once to observe the procedure is implemented correctly
env.run([agent_blocker, agent_random])
env.render(mode="ipython")

# Determine the winning percentages with
get_win_percentages(agent1=agent_blocker, agent2=agent_random)

# Submit to the competition
You may notice the following script is different to that in the course. Hopefully this is more intuitive.

First, we open our file "submission.py". Then, we search through our notebook and one-by-one add the functions needed for our agent. Finally we close our file.

Suppose we wanted to add our leftmost agent, we would use the following script.

In [ ]:
import inspect
import os

f = open("submission.py", "w")
f.write(inspect.getsource(agent_leftmost))
f.close()

print("agent_leftmost", "written to", "submission.py")

If, instead, we wish to submit our blocker agent, we would have to submit all functions associated with that agent. Notice that we must include our imports within the function so they are also written to the "submission.py" file.

In [ ]:
import inspect
import os

f = open("submission.py", "w")
f.write(inspect.getsource(drop_piece))
f.write(inspect.getsource(check_winning_move))
f.write(inspect.getsource(agent_blocker))
f.close()

print("agent_blocker", "written to", "submission.py")

To check this has worked, look in the tab to the right. Under data, find the "output" folder. Find your "submission.py" file and download it to observe the contents. We should have a self-contained script with all necessary functions for our agent. Now, we are ready to submit this solution!

# Submitting to the Competition
I want to refer back to the course here as it offers a very good explanation of how to make our final submission.

1. Begin by clicking on the blue Save Version button in the top right corner of the window. This will generate a pop-up window.
2. Ensure that the Save and Run All option is selected, and then click on the blue Save button.
3. This generates a window in the bottom left corner of the notebook. After it has finished running, click on the number to the right of the Save Version button. This pulls up a list of versions on the right of the screen. Click on the ellipsis (...) to the right of the most recent version, and select Open in Viewer. This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the Output tab on the right of the screen. Then, click on the blue Submit button to submit your results to the leaderboard.

And you've submitted!!! :)

I hope you all found success with this notebook. If you're still having difficulties submitting, feel free to comment on this notebook and I'll reply as soon as I can.